In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import codecs
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import  re
from gensim.models import Word2Vec
%matplotlib inline

In [17]:
train_path = 'train.csv'

In [19]:
def load_data(path):
    
    df = pd.read_csv(path, encoding = 'UTF-8')
    return df

In [164]:
df_train = load_data(train_path)

In [165]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [166]:
df_train = df_train.drop(['id', 'qid1', 'qid2'], axis=1)

In [167]:
df_train.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [38]:
def simpleFilter(sentence):

    filtered_sent = []
    sentence = re.sub(r'[^a-zA-Z0-9]', ' ', sentence)

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(sentence)
    

    for w in words:
            if w.lower() not in stop_words:
                    filtered_sent.append(w)
                    
#     filtered_sent = ' '.join(filtered_sent)

    return filtered_sent

In [39]:
def split_dataset(df, start, end):
    
    return df.iloc[start:end, :]

In [168]:
train = split_dataset(df_train, 0, 500)
val = split_dataset(df_train, 100001, 300000)

In [169]:
train['question1'] = train['question1'].apply(simpleFilter)
train['question2'] = train['question2'].apply(simpleFilter)

/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
train['question2'] = train['question2'].apply(simpleFilter)

/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [170]:
train.head()

,question1,question2,is_duplicate
0,"[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]",0
1,"[story, Kohinoor, Koh, Noor, Diamond]","[would, happen, Indian, government, stole, Koh...",0
2,"[increase, speed, internet, connection, using,...","[Internet, speed, increased, hacking, DNS]",0
3,"[mentally, lonely, solve]","[Find, remainder, math, 23, 24, math, divided,...",0
4,"[one, dissolve, water, quikly, sugar, salt, me...","[fish, would, survive, salt, water]",0


In [189]:
# vect = TfidfVectorizer(row_memory = False)
# question1 = vect.fit_transform(train['question1'])
# pd.DataFrame(question1.toarray(), columns = vect.get_feature_names())

In [4]:
def word2vector(question):
    
    model = Word2Vec(question, min_count=1)
    words = list(model.wv.vocab)
    vector = model[words]
    
    return vector

In [220]:
vec1 = train['question1'].apply(word2vector)

/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [221]:
vec2 = train['question2'].apply(word2vector)

/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [227]:
vec1[3].shape, vec2[3].shape

((10, 100), (14, 100))

In [245]:
vec1

0       [[0.0019871842, 0.001969118, -0.0035169523, -0...
1       [[0.0019862137, 0.001968448, -0.0035162575, -0...
2       [[7.019138e-05, -0.0035415792, 0.0042031636, -...
3       [[-0.0009757072, -0.0015634684, -0.00029498918...
4       [[0.0030159098, 0.0042842105, -0.0023776798, 0...
5       [[-0.0041321637, -0.0004437454, -0.004722553, ...
6       [[0.0020626767, 0.004181444, 0.002372044, 0.00...
7       [[-0.0020504738, -0.0012395128, 0.0011439148, ...
8       [[-0.002805937, 0.004540716, -0.0011508205, -0...
9       [[0.0010616853, 0.0018321011, 0.0044839666, 0....
10      [[0.0010616853, 0.0018321011, 0.0044839666, 0....
11      [[0.001151214, 0.0019150784, 0.0025018773, 0.0...
12      [[-0.0009757072, -0.0015634684, -0.00029498918...
13      [[0.0006146575, -0.004494815, -0.0041211834, 0...
14      [[0.00045915993, 0.004367285, -0.001088966, 0....
15      [[0.0006403471, -0.00097565906, 0.0018893208, ...
16      [[-0.0009757072, -0.0015634684, -0.00029498918...
17      [[-0.0

In [237]:
from sklearn.metrics import pairwise
s = pairwise.cosine_similarity(vec1[3],vec2[3], dense_output=True)
a = pd.DataFrame(s)

In [269]:
s

array([[-0.12667936,  0.1330758 ,  0.03522154, -0.2277365 , -0.05722146,
        -0.17128502,  0.9999999 , -0.09329993,  0.225535  , -0.03328187,
        -0.02893452, -0.01053553,  0.04611381, -0.10580146],
       [ 0.07207628, -0.00935888,  0.03748829,  0.12590565,  0.00871781,
         1.        , -0.17123109, -0.02890566, -0.16381088, -0.1075519 ,
         0.12129682,  0.09270362,  0.10340946, -0.10086416],
       [ 0.07719892, -0.12661088,  0.9999999 , -0.14491612, -0.09169605,
         0.03727191,  0.03516314,  0.01193368,  0.00141589, -0.21082988,
        -0.00661743,  0.11710373,  0.05062352, -0.15357873],
       [-0.01553059, -0.01878507,  0.00148561, -0.16031677, -0.06944671,
        -0.16385986,  0.22552617,  0.08349419,  1.        , -0.00752989,
        -0.05717545,  0.05337813,  0.0383581 , -0.31606826],
       [ 0.03908015, -0.03441771,  0.01190017,  0.14279673,  0.13567257,
        -0.0289435 , -0.09314851,  0.9999996 ,  0.0835311 , -0.08635464,
         0.14076054, -0.00

In [290]:
data = pd.DataFrame()
l = []
for i in range(s.shape[0]):
    for j in s:
        
        data['cs' + str(i)] = np.array(j)
#         print(j)

In [291]:
data

,cs0,cs1,cs2,cs3,cs4,cs5,cs6,cs7,cs8,cs9
0,0.151624,0.151624,0.151624,0.151624,0.151624,0.151624,0.151624,0.151624,0.151624,0.151624
1,-0.037180,-0.037180,-0.037180,-0.037180,-0.037180,-0.037180,-0.037180,-0.037180,-0.037180,-0.037180
2,-0.153998,-0.153998,-0.153998,-0.153998,-0.153998,-0.153998,-0.153998,-0.153998,-0.153998,-0.153998
3,0.029384,0.029384,0.029384,0.029384,0.029384,0.029384,0.029384,0.029384,0.029384,0.029384
4,0.051827,0.051827,0.051827,0.051827,0.051827,0.051827,0.051827,0.051827,0.051827,0.051827
5,-0.100717,-0.100717,-0.100717,-0.100717,-0.100717,-0.100717,-0.100717,-0.100717,-0.100717,-0.100717
6,-0.105792,-0.105792,-0.105792,-0.105792,-0.105792,-0.105792,-0.105792,-0.105792,-0.105792,-0.105792
7,0.114722,0.114722,0.114722,0.114722,0.114722,0.114722,0.114722,0.114722,0.114722,0.114722
8,-0.316068,-0.316068,-0.316068,-0.316068,-0.316068,-0.316068,-0.316068,-0.316068,-0.316068,-0.316068
9,-0.048597,-0.048597,-0.048597,-0.048597,-0.048597,-0.048597,-0.048597,-0.048597,-0.048597,-0.048597


In [282]:
type(data['cs0'][0])

numpy.float64

In [187]:
question1 = [train['question1'][1]]
print(question1)
model1 = Word2Vec(question1, min_count=1)
words = list(model1.wv.vocab)
print(words)
vector1 = model1[words]

# question2 = train['question2']
# model2 = Word2Vec(question2, min_count=1)
# vector2 = model2['sentence']

[['story', 'Kohinoor', 'Koh', 'Noor', 'Diamond']]
['story', 'Kohinoor', 'Koh', 'Noor', 'Diamond']


/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [244]:
print (vector1.shape)
print(vector1)

(5, 100)
[[ 3.5788614e-04 -3.1634402e-03  9.7066205e-04  1.9029808e-03
   4.6830550e-03 -4.2461039e-04 -1.6588887e-03  4.5354245e-04
   8.8091078e-04 -1.7946663e-04  1.3936843e-03 -2.8551803e-03
  -4.0526336e-04  3.8737350e-03  9.0062281e-04 -3.9840289e-03
   1.5258614e-03 -3.0916906e-03  4.6014176e-03  3.0462116e-03
  -1.9543439e-03 -2.4718316e-03  2.8281643e-03 -1.5164563e-03
   2.5484909e-03  3.2327513e-03  1.3732563e-03  1.8964956e-03
   4.4214986e-03 -3.8354676e-03  1.8174434e-03  4.4549215e-03
   4.2639202e-03 -4.9998420e-03 -4.2485758e-03 -2.1577924e-03
   1.6260018e-03  2.8679604e-03 -2.6868724e-03 -3.3963847e-03
  -3.4960825e-03  2.9206427e-03 -1.0360917e-03  7.1839509e-05
  -1.3018103e-03 -4.2439764e-03 -4.7994982e-03 -3.9620409e-03
  -4.5759813e-03 -4.4078743e-03 -1.2480839e-03 -7.6592329e-04
   3.0261849e-03  5.1923504e-04 -4.3704119e-03  4.5558857e-03
   4.2289686e-03  2.1104417e-03 -2.5520741e-03  2.1661776e-03
  -1.7653697e-03  4.7842953e-03 -4.4162185e-03 -3.1741902e-03

In [171]:
train['question1'][0]

['step', 'step', 'guide', 'invest', 'share', 'market', 'india']

In [152]:
dist = np.linalg.norm(vector1-vector2)
dist

0.26489913

In [156]:
print(model1)

Word2Vec(vocab=21406, size=100, alpha=0.025)


In [157]:
print(model2)

Word2Vec(vocab=20409, size=100, alpha=0.025)


In [158]:
vector1

array([ 0.06456608,  0.03256906, -0.0789848 ,  0.07845466, -0.05121044,
       -0.08446381, -0.12593625,  0.18131836, -0.03463574,  0.05487311,
       -0.09661291,  0.07371156,  0.11927063,  0.01375276, -0.03100643,
       -0.03012677,  0.0574383 ,  0.1166074 ,  0.08392926,  0.12399976,
        0.01111842, -0.00777644,  0.00229203,  0.13919373, -0.03406178,
        0.0511396 ,  0.07140026, -0.18719178, -0.07196861, -0.10793178,
       -0.07576232,  0.02158575, -0.05925721, -0.12358637, -0.06114413,
        0.0200294 ,  0.26763308, -0.11993156,  0.09369645,  0.01279841,
        0.01046564,  0.0545241 , -0.09027612, -0.00278676,  0.05947515,
        0.02290811, -0.0952461 , -0.09124961, -0.13434157,  0.1151384 ,
       -0.0483738 ,  0.09152637,  0.03570361, -0.01550004, -0.08927046,
       -0.10670144, -0.04639032,  0.10100814,  0.09929407, -0.09975764,
        0.07241693,  0.0530327 , -0.01438992, -0.06843679, -0.02262365,
        0.04167978, -0.02752242,  0.03039587, -0.19239524,  0.11

In [71]:
df_train['question2']

0                [step, step, guide, invest, share, market]
1         [would, happen, Indian, government, stole, Koh...
2                [Internet, speed, increased, hacking, DNS]
3         [Find, remainder, math, 23, 24, math, divided,...
4                       [fish, would, survive, salt, water]
5         [triple, Capricorn, Sun, Moon, ascendant, Capr...
6         [keeps, childern, active, far, phone, video, g...
7                                        [great, geologist]
8                                            [use, instead]
9                 [hack, Motorola, DCX3400, free, internet]
10        [things, technicians, tell, durability, reliab...
11                                 [see, Youtube, comments]
12                             [make, physics, easy, learn]
13                              [first, sexual, experience]
14        [laws, change, status, student, visa, green, c...
15        [Trump, presidency, affect, students, presentl...
16                                    [m

In [172]:
from gensim.models import Word2Vec
# define training data

sentences = []
for i in range(train.shape[0]):
    sentences.append(df_train['question1'][i] + df_train['question2'][i])
    
#print(df_train['question1'][0]+ df_train['question2'][0])
# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
# print(sentences)
# summarize vocabulary
#words = list(model.wv.vocab)
#print(words)
# access vector for one word
#print(model['words'])
# save model
#model.save('model.bin')
# load model
#new_model = Word2Vec.load('model.bin')
#print(new_model)

In [171]:
train.head()

,question1,question2,is_duplicate
0,"[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]",0
1,"[story, Kohinoor, Koh, Noor, Diamond]","[would, happen, Indian, government, stole, Koh...",0
2,"[increase, speed, internet, connection, using,...","[Internet, speed, increased, hacking, DNS]",0
3,"[mentally, lonely, solve]","[Find, remainder, math, 23, 24, math, divided,...",0
4,"[one, dissolve, water, quikly, sugar, salt, me...","[fish, would, survive, salt, water]",0


In [86]:
type(df_train.shape[0])

int

In [173]:
import numpy as np
from scipy import spatial

model = Word2Vec(sentences, min_count=1)
index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
#     words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
        print (feature_vec)
    return feature_vec

In [179]:
from gensim.matutils import cossim
sim = []
f1 = []
f2 = []
for i in range(train.shape[0]):
    s1_afv = avg_feature_vector(df_train['question1'][i], model=model, num_features=100, index2word_set=index2word_set)
    f1.append(s1_afv)
    s2_afv = avg_feature_vector(df_train['question2'][i], model=model, num_features=100, index2word_set=index2word_set)
    f2.append(s2_afv)
#     sim.append(1 - spatial.distance.cosine(s1_afv, s2_afv)) 
    sim.append(model.similarity(s1_afv, s2_afv))
# # print(sim)
#     sim = cossim(f1,f2)
    
    print(sim)

[ 3.82295577e-04  1.09277712e-03  1.32161425e-03  6.26321824e-04
  8.54891085e-04  2.47365353e-03  2.31200736e-03 -5.63635549e-04
  5.75752638e-04 -8.68648931e-04  1.76784664e-03  5.62354398e-04
  2.67387339e-04 -1.80263666e-03 -1.00544479e-03  8.87314323e-04
  1.42101708e-04 -1.17410731e-03 -9.26675217e-04  2.41298112e-04
  1.03800267e-03 -2.33562221e-03 -9.62568505e-04  5.66628238e-04
  1.54806461e-04 -1.63805904e-04 -4.43561585e-04 -3.34645039e-04
 -1.10245903e-03 -1.02144259e-03  7.37335242e-04 -1.66524795e-03
 -2.09507835e-03 -1.12523709e-03  2.22731163e-04 -2.33208435e-03
 -2.94659840e-04 -5.07081859e-05 -1.68602797e-04  1.32245987e-04
  1.82952697e-03  1.16040173e-03 -2.01004022e-03 -8.76717328e-04
  2.90417601e-03  6.98848395e-04  1.78534759e-03  1.27066392e-03
 -1.12601942e-04 -1.68194048e-04  1.99625254e-04 -1.13093760e-03
  3.19011509e-04 -1.15804852e-03  1.18720823e-03  1.24722405e-03
 -3.98771634e-04 -7.68968370e-04 -6.68385066e-04  1.79518887e-03
  1.19422970e-03 -7.01520

/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # This is added back by InteractiveShellApp.init_path()


KeyError: "word '0.00038229558' not in vocabulary"

In [100]:
train = train.drop(['question1', 'question2'], axis = 1)

In [118]:
y = train

In [120]:
def model_lr(df2,y2):
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression()
    lr.fit(df2,y2)
    return lr

In [ ]:
model(sim,)

In [113]:
f1 = pd.DataFrame(f1)
f2 = pd.DataFrame(f2)

In [125]:
X = pd.concat([f1,f2],axis = 1)

In [128]:
model_lr = model_lr(pd.concat([f1,f2],axis = 1), y)

/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [122]:
def log_loss(y2,y_pred):
    from sklearn.metrics import log_loss
    return log_loss(y2,y_pred)

In [124]:
def predict_proba(X, lr):
    y_pred_proba = lr.predict_proba(X)
    return y_pred_proba

In [155]:
X_test = df_train.iloc[400:500,:]

In [156]:
X_test['question1'] = X_test['question1'].apply(simpleFilter)
X_test['question2'] = X_test['question2'].apply(simpleFilter)

TypeError: expected string or bytes-like object

In [157]:
X_test.head()

,question1,question2,is_duplicate
400,"[could, start, open, source, development]","[start, contributing, open, source, projects]",1
401,"[dreams, look, real]","[dream, kind, harm, injuries, real, life]",0
402,"[cricket, players, really, drink, tea, tea, br...","[cricketers, actually, drink, tea, tea, break]",1
403,"[get, entry, MIT]","[requirements, selection, MIT]",1
404,"[fetus, breathe, inside, womb]","[fetus, exhale, womb]",1


In [158]:
sentences = []
for i in range(400, 500):
    sentences.append(X_test['question1'][i] + X_test['question2'][i])
    
#print(df_train['question1'][0]+ df_train['question2'][0])
# train model
model = Word2Vec(sentences, min_count=1)

In [159]:
sim = []
f3 = []
f4 = []
for i in range(400, 500):
    s1_afv = avg_feature_vector(X_test['question1'][i], model=model, num_features=100, index2word_set=index2word_set)
    f3.append(s1_afv)
    s2_afv = avg_feature_vector(X_test['question2'][i], model=model, num_features=100, index2word_set=index2word_set)
    f4.append(s2_afv)
    sim.append(1 - spatial.distance.cosine(s1_afv, s2_afv)) 

/home/sharad30/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


KeyError: "word 'step' not in vocabulary"